In [216]:
import spacy
import sys
import nltk
import regex as re

### Segregating the sentence into Subject predicate and object

In [303]:
def get_it_all_spacy(sen):
#     print(sen)
    doc = nlp(sen)
    count=0
    cclst = list()
    sublst = list()
    verblst = list()
    subjlst = list()
    objlst = list()
    for token in doc:
        count+=1
        ## Checks for the speech tag of word having a coordinating conjunction or "For" or "So"
        if "CC" in token.pos_ or " for " == " "+str(token)+" " or " so " == " "+str(token)+" ":
            cclst.append([count,str(token)])
    if len(cclst)==0:
        ## Append the coordinating conjunction along with its relative position
        cclst.append([count,"CCab"])
    count = 0
    for token in doc:
        count+=1
        if "VERB" in token.pos_ or "AUX" in token.pos_:
            ## Checks for speech tag of word having verb
            verblst.append([count,str(token)])
    if len(verblst)==0:
        ## Append the verb along with its relative position
        verblst.append([count,"VBab"])
    count = 0
    for token in doc:
        count+=1
        if "NOUN" in token.pos_ or "PRON" in token.pos_ or "PROPN" in token.pos_:
            sublst.append([count,str(token)])
    if len(sublst)==0:
        ## Append the noun along with its relative position
        sublst.append([count,"SBab"])
#     print("Subject = ",sublst)
#     print("Verb = ",verblst)
    k=0
    r=0
    i=0;
    flag = 0
    ccoccur = cclst[i][0]
    verboccur = verblst[k][0]   
    try:
        ## Use the verb of the relative position with relative noun.
        ## If the verb occurs before the noun then append as object else append as subject.
        ## but this is not necessarily the case everytime.. 
        ## I like playing basketball and I like playing football. 
        ## In ^ above case -- "I" that occurs before the verb "like" is taken as subject. 
        ## but for the same case "basketball" since it occurs after "like" becomes object.
        ## this is the simple case.. in the same example.. if the verb is still "like" of the 
        ## first clause, then "I" of the second clause is made object, hence it is necessary to locate
        ## the correct verb with respect to which the noun is taken. 
        ## the following code deals with locating the verb
        while verboccur< sublst[r][0]: # Checks for auxillary verbal conditions
            k+=1
            verboccur = verblst[k][0]
    except: # If no auxillary verb
        k-=1
        verboccur = verblst[k][0]
    try:    
        flag = 0
        while (1):
            j = sublst[r]
#             print("Accessing , ",j[1],verblst[k][1])
            ccoccur = cclst[i][0]
            verboccur = verblst[k][0]
            if j[0]<=verboccur: # Checks whether subject occurs before or after the verb.. and accordingly appends.
                subjlst.append([j[0],j[1]])
            if j[0]>verboccur:
                objlst.append([j[0],j[1]])
            r+=1 # Iterating to next noun, 
            try:
                j = sublst[r] # If next noun exists
            except:
                j = sublst[r-1] # If next noun doesnt exist
            if j[0]>=verboccur: # Locate the next verb if the next noun exists
                try:
                    while(j[0]>verboccur):
                        k+=1
                        verboccur = verblst[k][0]
                except: # reflect back to previous verb if the next verb was not successfully located
                    k-=1
                    verboccur = verblst[k][0]
#                 print(k,verblst[k])
            if(flag==1): # Terminating condition check
                try:
                    objlst.append([sublst[r][0],sublst[r][1]]) # If noun exists then last noun is object
                except:
#                     print("no last object")
                    pass
#                 print("broke")
                break;

            if(sublst[r][0]>=ccoccur): # Creating terminating condition... 
                try:
                    i+=1    
                    ccoccur = cclst[i][0] # If no forward coordinating conjunction, then no more breaks required.
                except:
                    i-=1
                    flag = 1; # Hence check one last time for clause and then break
#             print(ccoccur)

    except:
#         print(sys.exc_info()[0])
        pass
#     print(subjlst,objlst)
#     print(verblst)
#     print(cclst)
    return (subjlst,objlst,verblst,cclst)
#Data preparation completed
            

### Rule 1 segregation
##### -- Breaking conjunction


In [320]:
# Rule 1 segregation
# Breaking conjunction
def rule_1(sen):
    subjlst,objlst,verblst,cclst = get_it_all_spacy(sen)
    doc = nlp(sen)
#     print(subjlst,objlst,verblst,cclst)
    broke = list()
    i =0
    j=0
    count = len(cclst)
    if cclst[0][1] == "CCab" : # If no coordinating conjunction--already simple sentence
        return([],[],["Already Simple --- ",sen],1) # 1 == sends check to main to say "not a compound sentence"
    else:
        try:
            cap = cclst[i][1] # Locate the next coordinating conjunction and break the sentence further
            while (1):
#                 print(str(doc[j]))
    #             print(cap)
                cap = cclst[i][1]
                if str(cap).strip()==str(doc[j]):
    #                 print(str(doc[j]))
                    broke.append(j) # Append the location of the clauses to the broke list.
                    count-=1
                    if count!=0: # Check for multiple conjunctions
                        i+=1
                    else:
                        break
                j+=1
        except:
#             print(sys.exc_info()[0])
            pass
        # The following code appends the clause to the fin list depending on the broke locations.
        broke.append(len(str(doc))) 
#         print(broke)
        fin = list()
        fin.append(str(doc[0:broke[0]]))
        try:
            for i in range(len(broke)):
                fin.append(str(doc[broke[i]+1:broke[i+1]]))
        except:
#             print(sys.exec_info()[0])
            pass
    final= list()
    for i in fin:
    #     print(i)
        stri = str()    
        for j in i:
            stri+= j
        final.append(stri) # converting each word of the token to a string and appending the string
    broke = final
#     print(broke)
    for i in broke:
        if i == "": # if broke has any non existent string then remove.
            broke.remove(i)
    return (subjlst,objlst,broke,0) # Returns check as 0 to say "Compound sentence"

### Rule 2 or Rule 3 check
#### -- accordingly decompose

In [309]:
def rule_2_3(empsubjlst,allavail,onlyobjlst,onlysubjlst):
    count = 0
    try:
        # The following works on rule 2 or 3 check plus performs 
        # the rules on the lists provided it passes rule 1
        while len(empsubjlst)!=0: # If no subject then perform reverse rule 2
            # The following performs rule3
            extract = len(allavail)-1 # Gets the most recently used string to append to the subject. 
            subj3lst,obj3lst,verb3lst,cc3lst = get_it_all_spacy(allavail[extract][1])
            for i in empsubjlst:
                finstr= subj3lst[0][1]+" "+i[1] # appends the string to the subject
                count=subj3lst[0][0]+1
                allavail.append([count,finstr]) # appends the string to the list of simple sentences
#                 print(finstr)
                empsubjlst.remove(i) # removes the string from the incomplete sentence list
        while len(onlyobjlst)!=0: 
            # For normal rule2 usecase
            # extract finds the most recently used string
            # appends to that string the object
            # removes the incomplete clause and adds the complete simple sentence to the 
            # list of simple sentences.
            extract = len(allavail)-1
#             print(extract,len(allavail))
            subj3lst,obj3lst,verb3lst,cc3lst = get_it_all_spacy(allavail[extract][1])
            objlen=len(obj3lst)-1
            prestr = allavail[extract][1].replace(obj3lst[objlen][1],"")
        #     print(prestr)
            for i in onlyobjlst:
                finstr = prestr+" "+i[1]
                count=allavail[extract][0]+1
                allavail.append([count,finstr])
#                 print(finstr)
                onlyobjlst.remove(i)
        while len(onlysubjlst)!=0:
            # When only subject exists:
            # This is the reverse rule2 case:
            # gets the next complete sentence between which the subject exists.
            # appends this subject to the predicate and object of the corresponding found sentence
            # appends the final simple complete sentence to list of complete sentences
            # removes the incomplete sentence/ subject from the onlysubjlist.
            for i in onlysubjlst:
                extract = i[0]+1
                for j in range(len(allavail)):
                    if allavail[j][0] == extract:
                        extract = j
                        break
#                 print(extract)
                subj3lst,obj3lst,verb3lst,cc3lst = get_it_all_spacy(allavail[extract][1])
                subjlen = len(subj3lst)-1
                prestr = allavail[extract][1].replace(subj3lst[subjlen][1],"")
                finstr = i[1]+" "+prestr
                count=allavail[extract][0]+1
                allavail.append([count,finstr])
#                 print(finstr)
                onlysubjlst.remove(i)
        return (allavail)
    except:
#         print(sys.exc_info())
        pass



### Main function.. performs rule 1 and checks whether rule1 is sufficient else performs rule2 or 3 check

In [321]:
# Rule 1 check
def main(sen,fw):
    subjlst,objlst,broke,check = rule_1(sen)
#     print("\n------\n")
    ## Check denotes whether the sentence is already simple or not.
    if(check):
        fw.write("Sentence = "+sen+"\n")
        for i in broke:
            fw.write("\t"+i+"\n")
        fw.write("\n----\n")
        return
    ## The following lists are used to segregate rule 1 complete/incomplete..
    ## if incomplete then prepares data for rule2 and 3
    allavail = list()
    empsubjlst = list()
    empobjlst = list()
    empverblst = list()
    onlyobjlst = list()
    onlysubjlst = list()
    count=0;
    total = len(broke)
#     print(broke)
    for i in broke:
        ## Checks all sentences of the conjunction broke sentences. 
        subj=1;
        verb=1;
        obj=1;
        count+=1
        ## The following line gets the broken sentences predicate subject and object.
        subj2lst,obj2lst,verb2lst,cc2lst = get_it_all_spacy(i.strip())
#         print(subj2lst,obj2lst,verb2lst,cc2lst)
#         print(subjlst,objlst)
        ## if no subject: turn variable of subj 0 to denote.. no subject.
        if len(subj2lst)==0:
            subj=0
            if len(obj2lst)!=0:
                ## it is possible that object of the sentence may not be an object in the 
                ## original sentence before breaking by conjunction instead could be a subject
                ## in such case it finds the whether subject or object of original sentence
                ## and makes subj variable 1 and obj variable 0
                if any(obj2lst[0][1] in ls for ls in subjlst):
#                     print("Check",obj2lst[0][1],subjlst)
                    subj=1
                    obj=0
        if len(obj2lst)==0:
            obj=0;
            ## Similarly performs similar checking for subject of the sentence and makes 
            ## necessary variables show it. 
            if len(subj2lst)!=0:
                if any(subj2lst[0][1] in ls for ls in objlst):
#                     print("Check",subj2lst[0][1],objlst)
                    subj=0
                    obj=1
        ## If there is no verb in the sentence.. in case of dependent clauses. 
        if verb2lst[0][1]=="VBab":
            verb=0;
        if subj==0 and verb!=0 and obj!=0:
            ## empsubjlst is for the dependant clauses.. - rule 3
            empsubjlst.append([count,i])
        if subj==0 and verb==0 and obj!=0:
            ## onlyobjlst is for the independant clauses -- rule2
            onlyobjlst.append([count,i])
        if subj!=0 and verb!=0:
            ## allavail -- clauses that are simple and independent
            allavail.append([count,i])
        if subj!=0 and verb==0 and obj==0:
            ## For reverse rule 2 case where only subject exists in the dependant clause.
            onlysubjlst.append([count,i])
    # print(empsubjlst,onlysubjlst,onlyobjlst,allavail)
    # Check for case in Rule2 or Rule3:
    if len(broke)==len(allavail):
        fw.write("Sentence = "+sen+"\n")
        for i in allavail:
            print(re.sub(' +', ' ',i[1]).strip())
            fw.write("\t"+re.sub(' +', ' ',i[1]).strip()+"\n") 
        fw.write("\n-----\n")
    else:
        try:
            allavail = rule_2_3(empsubjlst,allavail,onlyobjlst,onlysubjlst)
            fw.write("Sentence = "+sen+"\n")
            for i in allavail:
                print(re.sub(r'\b(.+)\s+\1\b', r'\1', re.sub(' +', ' ',i[1]).strip()))
                fw.write("\t"+re.sub(r'\b(.+)\s+\1\b', r'\1', re.sub(' +', ' ',i[1]).strip())+"\n")
            fw.write("\n-----\n")
        except:
            fw.write("Sentence = "+sen+"\n")
#             print(sys.exc_info()[0])
            fw.write("\tAlready Simple\n")
            fw.write("\n-----\n")


#### Sepereate the sentences that have bare semi colon

In [324]:
def sep_semi(input):
    rw = open(input)
    fw = open("tempoutput.txt","w")
    for line in rw:
        line = re.sub("[^\P{P}\.;]|","",str(nlp(line)))
        fw.write(line+"\n")
#         r = line.split(". ")
#         for i in r:
#             if i==" ":
#                 r.remove(i)
# #         fw.write(line+"\n")
#         if(len(r)!=0):
#             for i in r:
#                 fw.write(i+"\n")
    rw.close()
    fw.close()
    rw = open("tempoutput.txt")
    lst = list()
    for line in rw:
        lst.append(line.split(";"))
    rw.close()
    rw = open("tempoutput.txt","w")
    for i in lst:
        if(len(i)>1):
            for sep in i:
                rw.write(sep.strip())
                rw.write("\n")
        else:
            rw.write(i[0])
#                 print(sep.strip())
    fw.close()
    rw.close()

### Function that coordinates with the main

In [325]:
nlp = spacy.load("en_core_web_sm")
# Splits sentences by the semi colon
# also removes punctuations
sep_semi(input("Enter the location of the input file: "))
rw = open("tempoutput.txt")
fw = open("output.txt","w")
for i in rw:
    main(i,fw)
    
rw.close()
fw.close()

Enter the location of the input file: input.txt
He went to the party
she stayed home.
He couldnt go home
he had no place to go.
I took a taxi
she drove home.
He didnt want help
did she offer it.
I wanted to go late
she wanted to go on time.
She cooked dinner
she went out to a restaurant.
She owned a car
she didnt know how to drive it.
She had to go
she called a friend to drive her.
I was tired from working late
I had to go to class anyway.
My brother just graduated from high school
he will attend St. Petersburg College.
I like peanut butter
I like jelly.
He eats macaroni
he wont eat cheese.
I like peanut butter
I like peanut jelly.
My family went to the park yesterday
we had a wonderful time.
Larrys family went to the zoo
they didnt enjoy it.
Your children are older
they probably wouldnt be scared.
Do you want to take them to the zoo
would it be wiser to go to the park
Lets go to the zoo
we can start with the monkeys
Susie both enjoy skiing.
Jim both enjoy skiing.
Bruce is efficient at